# <font color = "red"> Modelo de Classificação de Idiomas de Setenças com Bag of Words e Pytorch

In [1]:
!pip install -q -U torch torchvision

In [2]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch import nn, optim

## Preparação dos Dados

In [3]:
# Dados de Treino 
dados_treino = [("Tenho vinte paginas de leitura".lower().split(), "Portuguese"),
                ("I will visit the library".lower().split(), "English"),
                ("I am reading a book".lower().split(), "English"),
                ("This is my favourtie chapter".lower().split(), "English"),
                ("Estou na biblioteca lendo meu livro preferido".lower().split(), "Portuguese"),
                ("Gosto de livrios sobre viagens".lower().split(), "Portuguese")]

In [4]:
type(dados_treino)

list

In [5]:
dados_treino

[(['tenho', 'vinte', 'paginas', 'de', 'leitura'], 'Portuguese'),
 (['i', 'will', 'visit', 'the', 'library'], 'English'),
 (['i', 'am', 'reading', 'a', 'book'], 'English'),
 (['this', 'is', 'my', 'favourtie', 'chapter'], 'English'),
 (['estou', 'na', 'biblioteca', 'lendo', 'meu', 'livro', 'preferido'],
  'Portuguese'),
 (['gosto', 'de', 'livrios', 'sobre', 'viagens'], 'Portuguese')]

In [8]:
# Dados Teste
dados_teste = [("Estou lendo".lower().split(), "Portuguese"),
               ("this is not my favourite book".lower().split(), "English")]

In [10]:
# Prepara o dicionário do vocabulário

# Dicionário para o vocabulário 
dict_vocab = {}

# Contador 
i = 0

# Loop pelos dados de treino e teste
for palavras, idiomas in dados_treino + dados_teste:
 for palavra in palavras:
  if palavra not in dict_vocab:
   dict_vocab[palavra] = i
   i+=1


In [11]:
dict_vocab

{'tenho': 0,
 'vinte': 1,
 'paginas': 2,
 'de': 3,
 'leitura': 4,
 'i': 5,
 'will': 6,
 'visit': 7,
 'the': 8,
 'library': 9,
 'am': 10,
 'reading': 11,
 'a': 12,
 'book': 13,
 'this': 14,
 'is': 15,
 'my': 16,
 'favourtie': 17,
 'chapter': 18,
 'estou': 19,
 'na': 20,
 'biblioteca': 21,
 'lendo': 22,
 'meu': 23,
 'livro': 24,
 'preferido': 25,
 'gosto': 26,
 'livrios': 27,
 'sobre': 28,
 'viagens': 29,
 'not': 30,
 'favourite': 31}

In [12]:
# Numero de idiomas 
idiomas = 2

In [13]:
tamanho_corpus = len(dict_vocab)

In [14]:
tamanho_corpus

32

In [16]:
# Indice para os idiomas 
label_index = {"Portuguese":0, "English":1}

## <font color = "red">Construção do Modelo

In [20]:
# Classe para o modelo BOW de classificação 

class ModeloBOW(nn.Module): ## Classe recebe propriedades da classe mãe - "nn" do Pythorch

 # Método construtor
 def __init__(self, lista_idiomas, tamanho_corpus):
  super(ModeloBOW, self).__init__()
  self.linear = nn.Linear(tamanho_corpus, lista_idiomas) # Cria camada linear (multiplicação)

 # Feed Forward - Treinamento propriamente dito
 def forward(self, bow_vec): # Recebe o vetor do bag of words
  return F.log_softmax(self.linear(bow_vec), dim = 1)
 
 # "F" - pacote de funções de ativação do PyTorch

In [19]:
# Função para criar o vetor BOW necessário para o treinamento
def cria_bow_vetor(sentence, word_index):
 word_vec = torch.zeros(tamanho_corpus) # matriz de 0 com msm dim do corpus
 
 for word in sentence:
  word_vec[dict_vocab[word]] += 1
 return word_vec.view(1, -1)

In [21]:
# Função para criar a variável target
def cria_target(label, label_index):
 return torch.LongTensor([label_index[label]])

In [22]:
# Cria o Modelo
modelo = ModeloBOW(idiomas, tamanho_corpus)

In [23]:
# Função de perda (loss) - calculo negativo binomial
loss_function = nn.NLLLoss()

In [25]:
# Otimizador
optimizer = optim.SGD(modelo.parameters(), lr = 0.1) # lr = learning Rate
# Stocate Gradient Descent

## <font color = "red"> Treinamento do Modelo

In [26]:
# Loop de treinaemnto 

for epoch in range(100):
 
 for setence, label in dados_treino:

  modelo.zero_grad() # Inicia os Pesos
  
  bow_vec = cria_bow_vetor(setence, dict_vocab) # precisa do dicionario para criar o vetor
  target = cria_target(label, label_index) 

  log_probs = modelo(bow_vec) # calculando probaliblidade passando o vetor BOW

  loss = loss_function(log_probs, target) # calcula o erro verificando o target com as previsoes
  loss.backward()
  optimizer.step()

 if epoch % 10 == 0:
  print(f"Epoch: {str(epoch+1)}, Loss: {str(loss.item())}")

Epoch: 1, Loss: 0.44626864790916443
Epoch: 11, Loss: 0.07579302787780762
Epoch: 21, Loss: 0.04040587693452835
Epoch: 31, Loss: 0.027490265667438507
Epoch: 41, Loss: 0.0208209790289402
Epoch: 51, Loss: 0.01675274968147278
Epoch: 61, Loss: 0.014013230800628662
Epoch: 71, Loss: 0.012043262831866741
Epoch: 81, Loss: 0.010558576323091984
Epoch: 91, Loss: 0.00939968228340149


## <font color = "red">Previsões e Avaliação do Modelo

In [28]:
# Função para previsões
def faz_previsao(data):

 with torch.no_grad():
  setence = data[0]
  label = data[1]
  bow_vec = cria_bow_vetor(setence, dict_vocab)
  log_probs = modelo(bow_vec)
  print(setence)
  print(f"Probabilidade de ser o label {label} é igual a: {np.exp(log_probs)}")

In [29]:
# Previsão com a primeira setença de teste

faz_previsao(dados_teste[0])

['estou', 'lendo']
Probabilidade de ser o label Portuguese é igual a: tensor([[0.7819, 0.2181]])


In [30]:
dados_teste[0]

(['estou', 'lendo'], 'Portuguese')

In [31]:
dados_teste[1]

(['this', 'is', 'not', 'my', 'favourite', 'book'], 'English')

In [32]:
faz_previsao(dados_teste[1])

['this', 'is', 'not', 'my', 'favourite', 'book']
Probabilidade de ser o label English é igual a: tensor([[0.0284, 0.9716]])


## <font color = "red">Previsões com Novas Frases

In [63]:
# Dados Teste
novas_frases = [("Meu livro de biblioteca".lower().split(), "Portuguese"),
               ("I am visit".lower().split(), "English")]

In [64]:
novas_frases

[(['meu', 'livro', 'de', 'biblioteca'], 'Portuguese'),
 (['i', 'am', 'visit'], 'English')]

In [65]:
dados_treino

[(['tenho', 'vinte', 'paginas', 'de', 'leitura'], 'Portuguese'),
 (['i', 'will', 'visit', 'the', 'library'], 'English'),
 (['i', 'am', 'reading', 'a', 'book'], 'English'),
 (['this', 'is', 'my', 'favourtie', 'chapter'], 'English'),
 (['estou', 'na', 'biblioteca', 'lendo', 'meu', 'livro', 'preferido'],
  'Portuguese'),
 (['gosto', 'de', 'livrios', 'sobre', 'viagens'], 'Portuguese')]

In [66]:
faz_previsao(novas_frases[0])

['meu', 'livro', 'de', 'biblioteca']
Probabilidade de ser o label Portuguese é igual a: tensor([[0.9687, 0.0313]])


In [67]:
faz_previsao(novas_frases[1])

['i', 'am', 'visit']
Probabilidade de ser o label English é igual a: tensor([[0.0451, 0.9549]])
